# 1. Import Library

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料預處理

In [2]:
(x_Train, y_Train), (x_Test, y_Test) = mnist.load_data()

In [3]:
#將features以reshape轉換為60000*28*28*1的4維矩陣
x_Train4D=x_Train.reshape(x_Train.shape[0],28,28,1).astype('float32')
x_Test4D=x_Test.reshape(x_Test.shape[0],28,28,1).astype('float32')

In [4]:
#將features標準化
x_Train4D_normalize = x_Train4D / 255
x_Test4D_normalize = x_Test4D / 255

In [5]:
#One-hot encoding
y_TrainOneHot = np_utils.to_categorical(y_Train)
y_TestOneHot = np_utils.to_categorical(y_Test)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [7]:
#建立一個Sequential線性堆疊模型
model = Sequential()

In [8]:
#卷積層1
##16個濾鏡filter weight
##每個濾鏡大小5*5
##讓卷績運算產生的影像大小保持相同
##輸入的影像28*28，1表示單色灰階影響
##ReLU激活函數
model.add(Conv2D(filters=16,
                 kernel_size=(5,5),
                 padding='same',
                 input_shape=(28,28,1), 
                 activation='relu'))

In [9]:
#池化層1
##執行縮減取樣，將16個28*28的影像縮小為14*14
model.add(MaxPooling2D(pool_size=(2, 2)))

In [10]:
#卷積層2
##將原本16個影像轉換成36個
model.add(Conv2D(filters=36,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))

In [11]:
#池化層2
##執行第2次縮減取樣，將14*14的影像縮小為7*7
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#隨機在神經網路中放棄25%的神經元，避免overfitting
model.add(Dropout(0.25))

In [13]:
#平坦層
#36*7*7=1764個神經元
model.add(Flatten())

In [14]:
#隱藏層
#128個神經元
model.add(Dense(128, activation='relu'))

In [15]:
#隨機在神經網路中放棄50%的神經元，避免overfitting
model.add(Dropout(0.5))

In [16]:
#輸出層
#共10個神經元，用softmax激活函數進行轉換，將輸出轉換為預測每一個數字的機率
model.add(Dense(10,activation='softmax'))

In [17]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               225920    
__________

# 訓練模型

In [18]:
#用compile對模型進行設定
##loss:損失函數 cross_entropy
##optimizer:最優化方法 adam
##metrics:評估模型的方式 accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy']) 

In [ ]:
#用fit進行訓練
##x=x_Train4D_normalize:features數字影像的特徵值
##y=y_TrainOneHot:label數字影像真實的值
##validation_split:訓練/驗證比
##epochs:訓練週期次數
##batch_size:每批次數
##verbose:顯示訓練過程
train_history=model.fit(x=x_Train4D_normalize, 
                        y=y_TrainOneHot,validation_split=0.2, 
                        epochs=20, batch_size=300,verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20


In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
def show_train_history(train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
#訓練和驗證的準確率
show_train_history('acc','val_acc')

In [ ]:
#訓練和驗證的誤差
show_train_history('loss','val_loss')

# 評估模型準確率

In [ ]:
##x_Test4D_normalize:features
##y_TestOneHot:label
scores = model.evaluate(x_Test4D_normalize , y_TestOneHot)
scores[1]

# 預測結果

In [ ]:
prediction=model.predict_classes(x_Test4D_normalize)

In [ ]:
prediction[:10]

# 查看預測結果

In [ ]:
import matplotlib.pyplot as plt
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25 
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx], cmap='binary')

        ax.set_title("label=" +str(labels[idx])+
                     ",predict="+str(prediction[idx])
                     ,fontsize=10) 
        
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

In [ ]:
#顯示從0開始的10筆
plot_images_labels_prediction(x_Test,y_Test,prediction,idx=0)

# confusion matrix 混淆矩陣

In [ ]:
import pandas as pd
pd.crosstab(y_Test,prediction,
            rownames=['label'],colnames=['predict'])
#對角線是預測正確的，數值越高表示最不容易被混淆，越低則越容易
#非對角線表示預測錯誤

In [ ]:
df = pd.DataFrame({'label':y_Test, 'predict':prediction})

In [ ]:
#真實為5，但預測為3的
df[(df.label==5)&(df.predict==3)]